In [1]:
# ===== Cell 1: Setup =====
import os
from pathlib import Path

s3_uri = "s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/clothes_classifier_mobilenetv2.h5"

cwd = Path(os.getcwd())                       # current working directory
local_h5_path = cwd / "clothes_classifier_mobilenetv2.h5"
tfjs_output_dir = cwd / "tfjs_clothes_classifier"

print("CWD:", cwd)
print("Local .h5 will be saved to:", local_h5_path)
print("TFJS output dir:", tfjs_output_dir)


CWD: /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model
Local .h5 will be saved to: /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model/clothes_classifier_mobilenetv2.h5
TFJS output dir: /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model/tfjs_clothes_classifier


In [2]:
# ===== Cell 2: Download the .h5 from S3 into CWD =====
import re
import boto3

match = re.match(r"^s3://([^/]+)/(.+)$", s3_uri)
if match is None:
    raise ValueError(f"Invalid S3 URI: {s3_uri}")

bucket_name = match.group(1)
object_key = match.group(2)

s3_client = boto3.client("s3")
s3_client.download_file(bucket_name, object_key, str(local_h5_path))

print("Downloaded:", local_h5_path)
print("Exists:", local_h5_path.exists())
print("Bytes:", local_h5_path.stat().st_size)


Downloaded: /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model/clothes_classifier_mobilenetv2.h5
Exists: True
Bytes: 11507392


In [3]:
# ===== Cell 3: Load the Keras .h5 model =====
import tensorflow as tf

model = tf.keras.models.load_model(str(local_h5_path), compile=False)
model.summary()


2025-12-24 10:41:02.507449: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 10:41:02.509539: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-24 10:41:02.558410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-24 10:41:02.559200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-24 10:41:03.401981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [5]:
pip install tensorflow_hub

  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached h5py-3.15.1-cp310-cp310-linux_x86_64.whl
  Using cached keras-3.12.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 40.5 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.7 MB/s  0:00:00
Using cached keras-3.12.0-py3-none-any.whl (1.5 MB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.00/6 [typing-extensions]
  Attempting uninstall: h5py━━━━━━━━━━━━━━━━━━━━ 0/6 [typing-extensions]
    Found existing installation: h5py 3.8.0━ 0/6 [typing-extensions]
    Uninstalling h5py-3.

In [9]:
# ===== Cell X: Convert to TensorFlow.js format (with TFDF + JAX stubs) =====
import os
import shutil
from pathlib import Path

cwd = Path(os.getcwd())

# output folder in current working directory
tfjs_output_dir = cwd / "tfjs_clothes_classifier"

# Remove old output folder if it exists
if tfjs_output_dir.exists():
    shutil.rmtree(tfjs_output_dir)
tfjs_output_dir.mkdir(parents=True, exist_ok=True)

# -------------------------
# Stub #1: tensorflow_decision_forests (needed by tensorflowjs import chain)
# -------------------------
tfdf_stub = cwd / "tensorflow_decision_forests"
tfdf_stub.mkdir(exist_ok=True)
(tfdf_stub / "__init__.py").write_text(
    '"""Stub module to satisfy tensorflowjs import. Not used for Keras->TFJS conversion."""\n'
)

# -------------------------
# Stub #2: jax (needed by tensorflowjs.converters.jax_conversion import chain)
# We create: jax/experimental/jax2tf.py
# -------------------------
jax_stub = cwd / "jax"
(jax_stub / "experimental").mkdir(parents=True, exist_ok=True)

(jax_stub / "__init__.py").write_text(
    '"""Stub jax package to satisfy tensorflowjs import. Not used in Keras->TFJS conversion."""\n'
)

(jax_stub / "experimental" / "__init__.py").write_text(
    '"""Stub jax.experimental package."""\n'
)

(jax_stub / "experimental" / "jax2tf.py").write_text(
    '"""Stub jax.experimental.jax2tf module."""\n'
)

# IMPORTANT: ensure current directory is in sys.path first
import sys
if str(cwd) not in sys.path:
    sys.path.insert(0, str(cwd))

# Now import tensorflowjs
import tensorflowjs as tfjs

# Convert Keras model -> TFJS Layers format
tfjs.converters.save_keras_model(model, str(tfjs_output_dir))

print("TFJS output:", tfjs_output_dir)
print("Files:", sorted([p.name for p in tfjs_output_dir.iterdir()]))


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.


TFJS output: /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model/tfjs_clothes_classifier
Files: ['group1-shard1of3.bin', 'group1-shard2of3.bin', 'group1-shard3of3.bin', 'model.json']


In [10]:
# Upload local TFJS folder to S3 (recursive)

import os
import mimetypes
import boto3

bucket = "ai-bmi-predictor-v2"
s3_prefix = "tight and loose classifier/trained models/tfjs_clothes_classifier"  # folder in S3

# Update this if your folder lives somewhere else
local_tfjs_dir = os.path.join(os.getcwd(), "tfjs_clothes_classifier")

if not os.path.isdir(local_tfjs_dir):
    raise FileNotFoundError(f"Local folder not found: {local_tfjs_dir}")

s3 = boto3.client("s3")

uploaded = 0
for root, _, files in os.walk(local_tfjs_dir):
    for fname in files:
        local_path = os.path.join(root, fname)

        rel_path = os.path.relpath(local_path, local_tfjs_dir).replace("\\", "/")
        s3_key = f"{s3_prefix}/{rel_path}"

        content_type, _ = mimetypes.guess_type(local_path)
        extra_args = {}
        if content_type:
            extra_args["ContentType"] = content_type

        # Optional (often helpful for web assets):
        # extra_args["CacheControl"] = "public, max-age=31536000, immutable"

        if extra_args:
            s3.upload_file(local_path, bucket, s3_key, ExtraArgs=extra_args)
        else:
            s3.upload_file(local_path, bucket, s3_key)

        uploaded += 1
        print(f"Uploaded: s3://{bucket}/{s3_key}")

print(f"\nDone. Uploaded {uploaded} files from:\n  {local_tfjs_dir}")


Uploaded: s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/tfjs_clothes_classifier/group1-shard2of3.bin
Uploaded: s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/tfjs_clothes_classifier/group1-shard3of3.bin
Uploaded: s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/tfjs_clothes_classifier/model.json
Uploaded: s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/tfjs_clothes_classifier/group1-shard1of3.bin
Uploaded: s3://ai-bmi-predictor-v2/tight and loose classifier/trained models/tfjs_clothes_classifier/.ipynb_checkpoints/model-checkpoint.json

Done. Uploaded 5 files from:
  /home/ec2-user/SageMaker/tight and loose cloth classifier/javascript model/tfjs_clothes_classifier
